In [1]:
# read text sequence dataset
import pandas as pd
import numpy as np

train_seq_df = pd.read_csv("datasets/train/train_text_seq.csv")
train_seq_X = train_seq_df['input_str'].tolist()
train_seq_Y = train_seq_df['label'].tolist()
valid_seq_df = pd.read_csv('datasets/valid/valid_text_seq.csv')

test_seq_X = pd.read_csv("datasets/test/test_text_seq.csv")['input_str'].tolist()

In [2]:
# print(train_seq_df.head())
# print(train_seq_X[0])
train_seq_df['processed_str'] = train_seq_df['input_str']
valid_seq_df['processed_str'] = valid_seq_df['input_str']
from itertools import product

def remove_leading_zeros(seq):
    return seq.lstrip('0')
   
def generate_partitions(seq_length):
    part_lengths = [3, 4, 5]
    
    n = seq_length
    valid_partitions = []
    for partition in product(part_lengths, repeat=13):
        if sum(partition) == n:
            valid_partitions.append(partition)
    return valid_partitions
    
partition_dict = {}
for length in range(43, 51):
    partition_dict[length] = generate_partitions(length)

def check_pattern_conditions(parts):
    patterns = {
        '15436': 1,
        '1596': 2,
        '284': 1,
        '262': 2,
        '614': 2,
        '464': 1,
        '422': 1
    }
    
    pattern_counts = patterns.copy()
    
    
    for part in parts:
        for pattern in pattern_counts:
            if part == pattern:
                pattern_counts[pattern] -= 1
    
    
    return all(count == 0 for count in pattern_counts.values())


def remove_patterns(parts):
    patterns_to_remove = {'15436', '1596', '284', '262', '614', '464', '422'}
    pattern_counts = {'15436': 1, '1596': 2, '284': 1, '262': 2, '614': 2, '464': 1, '422': 1}
    
    remaining_parts = []
    
    for part in parts:
        if part in pattern_counts and pattern_counts[part] >0:
            pattern_counts[part] -= 1
            remaining_parts.append(".")
        else:
            remaining_parts.append(part)
    
    return remaining_parts


def process_sequence(seq):
   
    seq = remove_leading_zeros(seq)
    
    
    partitions = partition_dict[len(seq)]
    
   
    for partition_lengths in partitions:

        parts = []
        start = 0
        for length in partition_lengths:
            parts.append(seq[start:start+length])
            start += length
        
        if check_pattern_conditions(parts):
            
            remaining_seq = remove_patterns(parts)
            return remaining_seq
    

# # Example usage
# input_seq = "000015436159615961596284262262614614464422999999"
# result = process_sequence(input_seq)
# print("Remaining sequence:", result)


count = 0



remaining_parts_df = train_seq_df['processed_str'].apply(process_sequence).apply(pd.Series)

remaining_parts_df.columns = [f'part_{i+1}' for i in range(remaining_parts_df.shape[1])]

train_seq_df = pd.concat([train_seq_df, remaining_parts_df], axis=1)
remaining_parts_df2 = valid_seq_df['processed_str'].apply(process_sequence).apply(pd.Series)
remaining_parts_df2.columns = [f'part_{i+1}' for i in range(remaining_parts_df2.shape[1])]
valid_seq_df = pd.concat([valid_seq_df, remaining_parts_df2], axis=1)

# total_count = train_seq_df['input_str'].str.count("4262").sum()
# ss = process_sequence(train_seq_df['processed_str'][0])
# print(ss)

print(min(len(x) for x in train_seq_df['processed_str']))
print(max(len(x) for x in train_seq_df['processed_str']))


50
50


In [7]:
train_seq_df2 = train_seq_df.copy()
valid_seq_df2 = valid_seq_df.copy()
train_seq_df3 = train_seq_df.copy()
valid_seq_df3 = valid_seq_df.copy()

In [8]:
print(train_seq_df2.dtypes)

input_str        object
label             int64
processed_str    object
part_1           object
part_2           object
part_3           object
part_4           object
part_5           object
part_6           object
part_7           object
part_8           object
part_9           object
part_10          object
part_11          object
part_12          object
part_13          object
dtype: object


In [9]:
train_seq_df2.head()

,input_str,label,processed_str,part_1,part_2,part_3,part_4,part_5,part_6,part_7,part_8,part_9,part_10,part_11,part_12,part_13
0,0000154364642718159661428002624223132284159626...,0,0000154364642718159661428002624223132284159626...,.,.,2718,.,.,2800,.,.,3132,.,.,.,.
1,0004641596369515436422262614110471596262476161...,0,0004641596369515436422262614110471596262476161...,.,.,3695,.,.,.,.,11047,.,.,4761,.,.
2,0001543626215965999614422464135806142624051159...,0,0001543626215965999614422464135806142624051159...,.,.,.,5999,.,.,.,13580,.,.,4051,.,.
3,0000154364224641238614262159689561596284351061...,1,0000154364224641238614262159689561596284351061...,.,.,.,1238,.,.,.,8956,.,.,3510,.,.
4,0004641899422154362069015966142624761262159661...,1,0004641899422154362069015966142624761262159661...,.,1899,.,.,20690,.,.,.,4761,.,.,.,.


In [11]:
output_list = []
# print("Data types before conversion:")
# print(train_seq_df.dtypes)
# Iterate through each row in the DataFrame
final_parts = []
parts_dict = {}
parts_dict2 = {}
number_partitions = {}
for idx, row in train_seq_df2.iterrows():
    parts = row[3:]
    first_non_dot = None
    concatenated_str = ''
    count = 0
    for part in parts:
        if part != '.':
            if first_non_dot is None:
                first_non_dot = part  
            else:
                concatenated_str += part
        else:
            if first_non_dot is not None:
                if concatenated_str:
                    output_list.append(first_non_dot + concatenated_str)
                else:
                    count+= 1
                    if first_non_dot in parts_dict:
                        parts_dict[first_non_dot] += 1
                    else:
                        parts_dict[first_non_dot] = 1
                    final_parts.append(first_non_dot)
                first_non_dot = None
                concatenated_str = ''

    if first_non_dot is not None:
        if concatenated_str:
            # flag = 1
            output_list.append(first_non_dot + concatenated_str)
        else:
            count+=1
            final_parts.append(first_non_dot)
            if first_non_dot in parts_dict:
                parts_dict[first_non_dot] += 1
            else:
                parts_dict[first_non_dot] = 1
    if count == 0:
        number_partitions[output_list[len(output_list)-1]] = 3

    if count == 1:
        number_partitions[output_list[len(output_list)-1]] = 2



parts_dict2 = parts_dict.copy()
print(len(final_parts))
print(len(parts_dict))
print(len(parts_dict2))
print(parts_dict)

17004
224
224
{'2718': 138, '2800': 51, '3132': 133, '3695': 141, '11047': 50, '4761': 133, '5999': 139, '13580': 53, '4051': 142, '1238': 130, '8956': 36, '3510': 140, '1899': 130, '20690': 26, '33658': 53, '2919': 141, '2780': 125, '3365': 138, '8526': 42, '1495': 129, '30699': 42, '5705': 130, '2414': 133, '26390': 42, '2713': 132, '16510': 55, '5237': 137, '4790': 137, '20371': 52, '3720': 130, '18721': 41, '2154': 144, '3324': 149, '5103': 48, '3104': 140, '4309': 138, '11918': 34, '1828': 142, '12352': 38, '2598': 135, '3626': 54, '15611': 53, '8055': 36, '3682': 140, '1429': 44, '2670': 142, '3070': 134, '13956': 39, '6469': 136, '32600': 39, '2327': 134, '3388': 136, '3134': 140, '9102': 60, '2075': 134, '5066': 142, '670': 43, '5332': 121, '23684': 42, '2425': 135, '5163': 38, '4310': 141, '1415': 135, '4304': 139, '9831': 49, '2231': 134, '1433': 133, '22966': 43, '1983': 132, '3559': 131, '940': 143, '4427': 61, '3312': 133, '6761': 53, '3980': 137, '1795': 128, '1485': 137,

In [12]:
from itertools import product

def generate_partitions(seq_length,times):
    part_lengths = [3, 4, 5]
    
    n = seq_length
    valid_partitions = []
    
    for partition in product(part_lengths, repeat=times):
        if sum(partition) == n:
            valid_partitions.append(partition)

    return valid_partitions
    
    return valid_partitions
partition_dict = {}
for length in range(6, 15):
    partition_dict[length] = generate_partitions(length,2)
partition_dict2 = {}
for length in range(6, 15):
    partition_dict2[length] = generate_partitions(length,3)

In [13]:
def process_sequence(num,seq):
    
    partitions = []
    if num == 3:
        partitions = partition_dict2[len(seq)]
    else: 
        partitions = partition_dict[len(seq)]

    for partition_lengths in partitions:
        parts = []
        start = 0
        for length in partition_lengths:
            parts.append(seq[start:start+length])
            start += length
        # print(parts)
        for part in parts:
            if part in parts_dict:
                parts_dict[part] += 1
            else:
                parts_dict[part] = 1
                
for seq in output_list:
    process_sequence(number_partitions[seq],seq)
print(len(parts_dict))
print(len(parts_dict2))

2420
224


In [14]:
def process_sequence2(num,seq):

    partitions = []
    if num == 3:
        partitions = partition_dict2[len(seq)]
    else: 
        partitions = partition_dict[len(seq)]
    max_total = 0 
    best_partition = [] 
    for partition_lengths in partitions:
       
        parts = []
        start = 0
        total = 0  
        for length in partition_lengths:
            part = seq[start:start + length]
            parts.append(part)
            start += length
            # count = 0
      
        for part in parts:
            total += parts_dict[part] 
    
        if total >= max_total:
            max_total = total
            best_partition = parts
    # print(best_partition)
 
    return best_partition


In [15]:
partss = process_sequence2(number_partitions['43097002'],'43097002')
print(partss)

['4309', '7002']


In [16]:
store = {}
for seq in output_list:
    parts = process_sequence2(number_partitions[seq],seq)
    for part in parts:
        final_parts.append(part)
        if part in parts_dict2: 
            parts_dict2[part] += 1
        else:
            parts_dict2[part] = 1
    if seq not in store:
        store[seq] = []
        for part in parts:
            store[seq].append(part)
    
print(len(parts_dict))
print(len(parts_dict2))
print((store))


2420
232
{'80555237': ['8055', '5237'], '20914790': ['2091', '4790'], '360682091': ['36068', '2091'], '43097002': ['4309', '7002'], '31043901': ['3104', '3901'], '523724671': ['5237', '24671'], '207513956': ['2075', '13956'], '27181502': ['2718', '1502'], '38962327': ['3896', '2327'], '147383023': ['14738', '3023'], '196611157': ['19661', '1157'], '181031821': ['18103', '1821'], '462833980': ['46283', '3980'], '27183270': ['2718', '3270'], '307031505': ['3070', '31505'], '54495705': ['5449', '5705'], '28161923': ['2816', '1923'], '228172598': ['22817', '2598'], '299612506': ['2996', '12506'], '93883980': ['9388', '3980'], '271333658': ['2713', '33658'], '54495774': ['5449', '5774'], '19597118': ['1959', '7118'], '299645436': ['2996', '45436'], '229664310': ['22966', '4310'], '176453070': ['17645', '3070'], '24253720': ['2425', '3720'], '506632600': ['5066', '32600'], '327021832': ['3270', '21832'], '12382780': ['1238', '2780'], '13323070': ['1332', '3070'], '89672780': ['8967', '2780']

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense,Discretization, Dropout,SimpleRNN,GRU,Bidirectional,Input,Normalization
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense,Conv2D, Dropout,SimpleRNN,GRU,Bidirectional
from sklearn.metrics import accuracy_score
tokenizer = Tokenizer()
tokenizer.fit_on_texts(final_parts)

In [20]:
print(tokenizer.word_index)


{'3510': 1, '2670': 2, '5066': 3, '5774': 4, '4869': 5, '2414': 6, '2999': 7, '2857': 8, '1959': 9, '1120': 10, '3132': 11, '3324': 12, '940': 13, '4521': 14, '3023': 15, '2919': 16, '3682': 17, '1415': 18, '1795': 19, '2931': 20, '2548': 21, '1129': 22, '2791': 23, '2718': 24, '3695': 25, '4051': 26, '5237': 27, '3104': 28, '4309': 29, '4310': 30, '1314': 31, '1821': 32, '2481': 33, '2091': 34, '2920': 35, '486': 36, '5999': 37, '1828': 38, '2598': 39, '2327': 40, '4304': 41, '1485': 42, '4524': 43, '6659': 44, '2816': 45, '4349': 46, '2078': 47, '3720': 48, '6469': 49, '2425': 50, '1433': 51, '3980': 52, '1731': 53, '2682': 54, '2996': 55, '1270': 56, '4761': 57, '2154': 58, '3134': 59, '1065': 60, '5333': 61, '1157': 62, '4790': 63, '3070': 64, '3388': 65, '2075': 66, '3312': 67, '3901': 68, '3270': 69, '1558': 70, '2780': 71, '1983': 72, '1507': 73, '1495': 74, '5705': 75, '2713': 76, '5332': 77, '2231': 78, '3553': 79, '1954': 80, '3559': 81, '2998': 82, '1238': 83, '1899': 84, '3

In [21]:
data = []
for idx, row in train_seq_df2.iterrows():
    parts = row[3:]  
    first_non_dot = None
    concatenated_str = ''
    parts2 = []
    
    # Iterate through each part
    for part in parts:
        if part != '.':
            if first_non_dot is None:
                first_non_dot = part  
            else:
                concatenated_str += part
        else:
            if first_non_dot is not None:
                if concatenated_str:
                    joined_part = first_non_dot+concatenated_str
                    for pp in store[joined_part]:
                        parts2.append(pp)
                else:
                    parts2.append(first_non_dot)
                first_non_dot = None
                concatenated_str = ''
    if first_non_dot is not None:
        if concatenated_str:
            joined_part = first_non_dot+concatenated_str
            for pp in store[joined_part]:
                parts2.append(pp)
        else:
            parts2.append(first_non_dot)
    data.append({'input_str' : row['input_str'],'label':row['label'], 'part_1':parts2[0], 'part_2':parts2[1],'part_3':parts2[2]})


final_df = pd.DataFrame(data)
final_df.head()



,input_str,label,part_1,part_2,part_3
0,0000154364642718159661428002624223132284159626...,0,2718,2800,3132
1,0004641596369515436422262614110471596262476161...,0,3695,11047,4761
2,0001543626215965999614422464135806142624051159...,0,5999,13580,4051
3,0000154364224641238614262159689561596284351061...,1,1238,8956,3510
4,0004641899422154362069015966142624761262159661...,1,1899,20690,4761


In [31]:
final_df[['part_1', 'part_2', 'part_3']] = final_df[['part_1', 'part_2', 'part_3']].astype(str)
df_top3 = final_df[['part_1', 'part_2', 'part_3']].copy()


tokenized_part_1 = tokenizer.texts_to_sequences(df_top3['part_1'])
tokenized_part_2 = tokenizer.texts_to_sequences(df_top3['part_2'])
tokenized_part_3 = tokenizer.texts_to_sequences(df_top3['part_3'])
tokenized_sequences = [part_1 + part_2+ part_3 for part_1, part_2, part_3 in zip(tokenized_part_1, tokenized_part_2, tokenized_part_3)]

# Convert the list to a numpy array
tokenized_array = np.array(tokenized_sequences)
X_padded = tokenized_array
# Output the final tokenized array
print("Tokenized Array Shape:", tokenized_array.shape)
print("Sample Tokenized Sequences:")

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(final_df['label'])
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)


max_len = 3
model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=50, input_length=max_len, trainable=False))

model.add(Conv1D(filters=48, kernel_size=1, activation='relu'))

model.add((SimpleRNN(20, return_sequences=False)))


model.add(Dense(64, activation='relu',use_bias=True))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=128, verbose=1)

y_pred2 = (model.predict(X_test) > 0.5).astype(int)


accuracy2 = accuracy_score(y_test,y_pred2)
print(f"Neural Network Test Accuracy: {accuracy2 * 100:.2f}%")
model.summary()
print(X_padded[:5])
print(X_train[0])
print(X_train[1])

Tokenized Array Shape: (7080, 3)
Sample Tokenized Sequences:
Epoch 1/30


/home/baftol/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5087 - loss: 0.6926
Epoch 2/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6536 - loss: 0.6625
Epoch 3/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7512 - loss: 0.5263
Epoch 4/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8261 - loss: 0.3891
Epoch 5/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8943 - loss: 0.2714
Epoch 6/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9240 - loss: 0.1992
Epoch 7/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9479 - loss: 0.1536
Epoch 8/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9547 - loss: 0.1277
Epoch 9/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9538 - loss: 0.1192
Epoch 10/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9593 - loss: 0.1073
Epoch 11/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9687 - loss: 0.0896
Epoch 12/30
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9740 - loss: 0.0760


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ (None, 3, 50)          │        11,650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 3, 48)          │         2,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 20)             │         1,380 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,363 (106.89 KB)

 Trainable params: 5,237 (20.46 KB)

 Non-trainable params: 11,650 (45.51 KB)

 Optimizer params: 10,476 (40.93 KB)

[[ 24 120  11]
 [ 25 104  57]
 [ 37  96  26]
 [ 83 191   1]
 [ 84 202  57]]
[ 79 198  64]
[ 49 135  55]
